In [1]:
import os,gc, pandas as pd
pd.set_option('display.max_columns', 500)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/infer_data/test.parquet')
df.ts = (df.ts//1000).astype('int32')

mn = df.ts.min()
#df['hour'] = df.ts % (60*60*24)
df['day'] = (df.ts-mn)  // (60*60*24) #% (60*60*24*7)
print( df.shape )
df.head()

(6928123, 5)


,session,aid,ts,type,day
0,12899779,59625,1661724000,0,0
1,12899780,1142000,1661724000,0,0
2,12899780,582732,1661724058,0,0
3,12899780,973453,1661724109,0,0
4,12899780,736515,1661724136,0,0


In [3]:
AIDS = df.aid.unique().values
DAYS = df.day.unique().values

In [4]:
DAYS

array([0, 1, 2, 3, 4, 5, 6])

In [5]:
tmp = df.groupby(['aid','day']).agg({'aid':'count','session':'count'})
tmp.columns = ['item','user']

In [6]:
tmp.head()

,,item,user
aid,day,,
682985,1,1,1
851790,1,1,1
430207,6,6,6
302948,5,1,1
80956,6,2,2


In [7]:
import cupy
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,len(DAYS)).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')
dff = dff.set_index( ['aid','day'] )

In [8]:
dff = dff.merge(tmp, left_index=True, right_index=True, how='left').fillna(0).reset_index()

In [9]:
dff.head()

,aid,day,item,user
0,3637,2,1,1
1,3637,5,1,1
2,3641,0,1,1
3,3642,1,1,1
4,3637,1,0,0


In [10]:
dff.loc[dff.aid==0]

,aid,day,item,user
17824,0,0,1,1
17825,0,5,1,1
17826,0,6,2,2
17834,0,1,0,0
17835,0,2,0,0
17836,0,3,0,0
17837,0,4,0,0


In [11]:
item_features3 = dff.groupby('aid').agg({'item':'std','user':'std'}).fillna(-1)
item_features3.columns = ['item_day_std','user_day_std']

f32 = ['item_day_std','user_day_std']
for c in f32: item_features3[c] = item_features3[c].astype('float32')

In [12]:
item_features3.head()

,item_day_std,user_day_std
aid,,
711749,0.377964,0.377964
1098181,1.253566,1.253566
1619389,7.892130,7.892130
509821,1.718249,1.718249
859329,1.000000,1.000000


In [13]:
item_features3.loc[0]

item_day_std    0.786796
user_day_std    0.786796
Name: 0, dtype: float32

In [14]:
del dff
gc.collect()

43

In [15]:
df.head()

,session,aid,ts,type,day
0,12899779,59625,1661724000,0,0
1,12899780,1142000,1661724000,0,0
2,12899780,582732,1661724058,0,0
3,12899780,973453,1661724109,0,0
4,12899780,736515,1661724136,0,0


In [16]:
item_features = df.groupby(['aid','day']).agg({'type':'mean','aid':'count','session':'nunique'})
item_features.columns = ['buy_ratio','count_item','count_user']

f32 = ['buy_ratio']
for c in f32: item_features[c] = item_features[c].astype('float32')
    
i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [17]:
item_features.head()

buy_ratio  count_item  count_user
aid day                                   
0   0      0.00000           1           1
    5      0.00000           1           1
    6      0.00000           2           2
3   0      0.10000          80          47
    1      0.04918          61          34

In [18]:
dff = cudf.DataFrame(columns=['aid','day'])
dff['aid'] = cupy.repeat(AIDS,len(DAYS)).astype('int32')
dff['day'] = cupy.tile(DAYS,len(AIDS)).astype('int8')
dff = dff.set_index( ['aid','day'] )

In [19]:
dff = dff.merge(item_features, left_index=True, right_index=True, how='left').fillna(-1).reset_index()
dff = dff.sort_values(['aid','day'])

In [20]:
dff2 = cudf.DataFrame(index=AIDS)
for c in ['buy_ratio','count_item','count_user']:
    v = dff[c].values.reshape((-1,7))
    for k in range(7):
        n = f'{c}_{k}'
        dff2[n] = v[:,k] 

In [21]:
del dff
gc.collect()

0

In [22]:
dff2.head()

,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6
0,0.0,-1.00000,-1.000000,-1.0,-1.0,0.0,0.0,1,-1,-1,-1,-1,1,2,1,-1,-1,-1,-1,1,2
3,0.1,0.04918,0.259259,0.0,0.0,0.0,-1.0,80,61,54,5,1,1,-1,47,34,30,4,1,1,-1
4,-1.0,0.00000,0.000000,-1.0,-1.0,0.0,0.0,-1,1,1,-1,-1,4,2,-1,1,1,-1,-1,2,1
6,0.0,-1.00000,-1.000000,-1.0,-1.0,-1.0,-1.0,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1
7,-1.0,-1.00000,-1.000000,0.0,-1.0,-1.0,-1.0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [23]:
dff2.dtypes

buy_ratio_0     float32
buy_ratio_1     float32
buy_ratio_2     float32
buy_ratio_3     float32
buy_ratio_4     float32
buy_ratio_5     float32
buy_ratio_6     float32
count_item_0      int32
count_item_1      int32
count_item_2      int32
count_item_3      int32
count_item_4      int32
count_item_5      int32
count_item_6      int32
count_user_0      int32
count_user_1      int32
count_user_2      int32
count_user_3      int32
count_user_4      int32
count_user_5      int32
count_user_6      int32
dtype: object

In [24]:
print( dff2.shape )
dff2.head()

(783486, 21)


,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6
0,0.0,-1.00000,-1.000000,-1.0,-1.0,0.0,0.0,1,-1,-1,-1,-1,1,2,1,-1,-1,-1,-1,1,2
3,0.1,0.04918,0.259259,0.0,0.0,0.0,-1.0,80,61,54,5,1,1,-1,47,34,30,4,1,1,-1
4,-1.0,0.00000,0.000000,-1.0,-1.0,0.0,0.0,-1,1,1,-1,-1,4,2,-1,1,1,-1,-1,2,1
6,0.0,-1.00000,-1.000000,-1.0,-1.0,-1.0,-1.0,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1
7,-1.0,-1.00000,-1.000000,0.0,-1.0,-1.0,-1.0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1


In [25]:
print( item_features3.shape )
item_features3.head()

(783486, 2)


,item_day_std,user_day_std
aid,,
711749,0.377964,0.377964
1098181,1.253566,1.253566
1619389,7.892130,7.892130
509821,1.718249,1.718249
859329,1.000000,1.000000


In [26]:
import cupy, numpy as np
A = cupy.asnumpy( dff2.index.values )
B = cupy.asnumpy( item_features3.index.values)

In [27]:
np.setdiff1d(B,A)

array([], dtype=int32)

In [28]:
item_features3.loc[0]

item_day_std    0.786796
user_day_std    0.786796
Name: 0, dtype: float32

In [29]:
#item_features = cudf.concat([dff2,item_features3],axis=1)

item_features = dff2.merge(item_features3, left_index=True, right_index=True, how='left')

In [30]:
item_features.head()

,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6,item_day_std,user_day_std
5576,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1,-1,-1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,2,0.755929,0.755929
5579,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,0.377964,0.377964
5580,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1,1,2,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,0.786796,0.786796
5582,-1.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,1,-1,-1,-1,-1,-1,-1,1,-1,-1,-1,-1,-1,0.377964,0.377964
5584,-1.0,-1.0,0.0,-1.0,-1.0,0.0,-1.0,-1,-1,1,-1,-1,1,-1,-1,-1,1,-1,-1,1,-1,0.487950,0.487950


In [31]:
item_features.loc[ [0,3,4] ]

,buy_ratio_0,buy_ratio_1,buy_ratio_2,buy_ratio_3,buy_ratio_4,buy_ratio_5,buy_ratio_6,count_item_0,count_item_1,count_item_2,count_item_3,count_item_4,count_item_5,count_item_6,count_user_0,count_user_1,count_user_2,count_user_3,count_user_4,count_user_5,count_user_6,item_day_std,user_day_std
0,0.0,-1.00000,-1.000000,-1.0,-1.0,0.0,0.0,1,-1,-1,-1,-1,1,2,1,-1,-1,-1,-1,1,2,0.786796,0.786796
3,0.1,0.04918,0.259259,0.0,0.0,0.0,-1.0,80,61,54,5,1,1,-1,47,34,30,4,1,1,-1,34.724770,34.724770
4,-1.0,0.00000,0.000000,-1.0,-1.0,0.0,0.0,-1,1,1,-1,-1,4,2,-1,1,1,-1,-1,2,1,1.463850,1.463850


In [32]:
item_features.item_day_std.isna().sum()

0

In [33]:
item_features.columns = [f'{c}_6' for c in item_features.columns]

In [34]:
item_features.dtypes

buy_ratio_0_6     float32
buy_ratio_1_6     float32
buy_ratio_2_6     float32
buy_ratio_3_6     float32
buy_ratio_4_6     float32
buy_ratio_5_6     float32
buy_ratio_6_6     float32
count_item_0_6      int32
count_item_1_6      int32
count_item_2_6      int32
count_item_3_6      int32
count_item_4_6      int32
count_item_5_6      int32
count_item_6_6      int32
count_user_0_6      int32
count_user_1_6      int32
count_user_2_6      int32
count_user_3_6      int32
count_user_4_6      int32
count_user_5_6      int32
count_user_6_6      int32
item_day_std_6    float32
user_day_std_6    float32
dtype: object

In [35]:
item_features.to_parquet('../../data/item_user_features/item13_LB.pqt')